In [1]:
%reload_ext autoreload
%autoreload 2

#make data file

import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import time
from suite2p.run_s2p import run_s2p



# Get the list of recordings
info = mfun.analysis()
# display the detected session
print( "Total Session: " +  str(info.recordingList .shape[0]))
#info.recordingList.head()


Env: suite2p
No module named 'cellpose'
cannot use anatomical mode, but otherwise suite2p will run normally
Computer: Huriye Windows


In [ ]:
# default settings for analysis
ops = {
        'batch_size': 200, # reduce if running out of RAM
        'fast_disk': os.path.expanduser('~/suite2p_binaries'), # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
         #'save_path0': '/media/jamesrowland/DATA/plab/suite_2p', # stores results, defaults to first item in data_path
        'delete_bin': False, # whether to delete binary file after processing
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'diameter': 12, # this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
        'tau':  1.26, # this is the main parameter for deconvolution (1.25-1.5 for gcamp6s)
        'fs': 30.,  # sampling rate (total across planes)
        # output settings
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        # parallel settings
        'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
        'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
        # registration settings
        'do_registration': True, # whether to register data
        'nimg_init': 200, # subsampled frames for finding reference image
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': True, # whether to save registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        # cell detection settings
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'navg_frames_svd': 5000, # max number of binned frames for the SVD
        'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'ratio_neuropil': 6., # ratio between neuropil basis size and cell radius
        'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
        'tile_factor': 1., # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
        'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'outer_neuropil_radius': np.inf, # maximum neuropil radius
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        # deconvolution settings
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
      }


In [ ]:
# Check Suite2p data for learning phase - Each animal should have one suite2p file. 
# needs to be run in a different enviroment : suite2p
# Extraction needs to be done with Matlab
animalList = ['OFZ008']# np.unique(info.recordingList.animalID)
for animal in animalList:
    print('Starting suite2p for ' + animal)
    filtered_sessionOneAnimal = info.recordingList[(info.recordingList['animalID'] == animal) &
                                                    (info.recordingList['twoP'] == True) & 
                                                    (info.recordingList['ROI'] ==) ]
    filtered_sessionOneAnimal = filtered_sessionOneAnimal.reset_index(drop=True)
    roiList = np.unique(filtered_sessionOneAnimal.ROI)
    for roi in roiList:
        print(animal + ' ROI:' + str(roi))
        filtered_session = filtered_sessionOneAnimal[(filtered_sessionOneAnimal['ROI'] == roi)] 
        filtered_session = filtered_session.reset_index(drop=True)
        # run suite2p
        # make the local suite2p binaries file if it does not already exist
        save_folder =   os.path.join(info.suite2pOutputPath, animal, 'suite2p_ROI-' + str(roi).zfill(3))
        runnedBefore = False
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        checkFileExist = [f for f in glob.glob(save_folder)]
        if len(checkFileExist)>0:
            runnedBefore = True
        # run before if condition should be here for full automation
        if ~runnedBefore:
            tiff_list =[]
            for idx, sessionName in enumerate(filtered_session.sessionName):
                twoP_path  = os.path.join(info.rawPath, animal,
                                         filtered_session.recordingDate[idx],'TwoP')
                recording_id = filtered_session.recordingID[idx]
                ses_path = glob.glob(twoP_path+ '\*t-' +str(recording_id).zfill(3))
                ses_path = ses_path[0]
                ses_name = [f for f in glob.glob(ses_path + '\*tif')]
                # For debugging the files
                # if animal == 'OFZ008'or animal == 'OFZ009' :
                #    print(ses_path)
                #    print(ses_name)
                ses_path = glob.glob(twoP_path+'\*t-' +str(recording_id).zfill(3))
                ses_path = ses_path[0]
                tiff_list.append(os.path.join(twoP_path,
                                         filtered_session.recordingDate[idx]+'_'+ animal + '_t-', str(recording_id).zfill(3),
                                         ses_name[0]))
            print('Number of session detected: ' + str(len(tiff_list)))
            print(tiff_list)
            db = { 
                'data_path':  os.path.join(info.rawPath, animal),
                'tiff_list': tiff_list,
                'save_folder': save_folder
                }
             
            t1 = time.time()
            opsEnd = run_s2p(ops=ops,db=db)
            t2 = time.time()
            print( animal + ' : ROI - ' + str(roi) + ': Suite2p is created in {}'.format(t2 - t1) + 'seconds.')
        else:
            print( animal + ' : ROI - ' + str(roi) + ': Suite2p is created before')
print('Completed')
